In [1]:
!pip install python-schema-registry-client

from pyspark.sql.avro.functions import from_avro
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.functions import to_date
import base64
import gzip
import _datetime
from schema_registry.client import SchemaRegistryClient
import json
import sys
import datetime
from pyspark.sql.functions import max as sparkMax
from pyspark.sql.functions import broadcast

# import org.apache.spark.sql.streaming.ProcessingTime

In [2]:
from pyspark.sql import SparkSession

# spark = SparkSession.\
#         builder.\
#         appName("pyspark-notebook").\
#         master("spark://spark-master:7077").\
#         config("spark.executor.memory", "512m").\
#         getOrCreate()


# org.apache.spark:spark-avro:3.0.0

# SparkConf conf = new SparkConf()
#             .setAppName("Test spark")
#             .setMaster("spark://ip of your master node:port of your master node")
#             .set("spark.blockManager.port", "10025")
#             .set("spark.driver.blockManager.port", "10026")
#             .set("spark.driver.port", "10027") //make all communication ports static (not necessary if you disabled firewalls, or if your nodes located in local network, otherwise you must open this ports in firewall settings)
#             .set("spark.cores.max", "12") 
#             .set("spark.executor.memory", "2g")
#             .set("spark.driver.host", "ip of your driver (PC)");
        
    
import os

# setup arguments
os.environ['kafka1'] = '34.220.52.64'
os.environ['kafka2'] = '52.35.125.165'
os.environ['schema-registry'] = '52.35.125.165'


os.environ['PYSPARK_PYTHON'] = "/opt/workspace/env3/bin/python3.9"

spark = SparkSession.builder.appName('pyspark-notebook')\
    .master("spark://spark-master:7077")\
    .config('spark.jars.packages', 'com.amazonaws:aws-java-sdk-bundle:1.11.974,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0,org.apache.spark:spark-sql_2.12:3.0.0,org.apache.kafka:kafka-clients:2.8.0,org.apache.spark:spark-avro_2.12:3.0.0,io.delta:delta-core_2.12:0.8.0,org.apache.spark:spark-streaming-kafka-0-10_2.12:3.0.0,org.apache.spark:spark-streaming_2.12:3.0.0')\
    .config("spark.executor.memory", "512m")\
    .config("spark.jars", "scalaSpark-assembly-1.0.jar") \
    .config("spark.cores.max", "12")\
    .config("spark.driver.port", "4040")\
    .config("spark.archives","venv1.zip")\
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .getOrCreate()

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-streaming_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0c3ffae7-2c3c-4833-a234-a66d606b0dc4;1.0
	confs: [default]
	found com.amazonaws#aws-java-sdk-bundle;1.11.974 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-pr

In [3]:
spark.sparkContext.addPyFile("scalaSpark-assembly-1.0.jar")

In [4]:
from delta.tables import *
import pyspark.sql.functions as fn
data = spark.range(1,5)
data.write.format("delta").mode("overwrite").save("delta_sample2")

22/03/07 12:21:08 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
22/03/07 12:21:08 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:716)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:152)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:258)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:168)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:203)
	at org.apache.spark.rpc.netty.Inbox.process(Inbox.scala:100)
	at org.apache.spark.rpc.netty.MessageLoop.org$apache$spark$rpc$netty$Mess

In [2]:
# df4 = spark.sparkContext._jvm.com.example.Hello.add

# print(spark._jsparkSession)
df5 = spark.sparkContext._jvm.com.example.Hello.add(spark._jsparkSession, "34.220.52.64:9092,52.35.125.165:9092", "datalake-qa-91")

datalake-qa-91

This method get is used to extract columns present in schema and generate sql statements for create , update and insert table

In [8]:
def get_partition_query(partition_column: str, primary_key: str):

        if not primary_key or not partition_column:
            return "updatesTable.id = targetTable.id"
        
#         query = "updatesTable.{0} = targetTable.{0}".format(primary_key)
        
        query = ""
        
        primary_keyA = []
    
        pkey_array = primary_key.split(",")
        for partitions in pkey_array:
            partitions = partitions.strip()
            if query == "":
              query += "updatesTable.{0} = targetTable.{0}".format(partitions)
            else:
              query += " and updatesTable.{0} = targetTable.{0}".format(partitions)
              
            primary_keyA.append(partitions)
        
#         primary_key = primary_key.strip()
        partitions_array = partition_column.split(",")
        for partitions in partitions_array:
            partitions = partitions.strip()
            if partitions not in primary_keyA:
                query += " and updatesTable.{0} = targetTable.{0}".format(partitions)
        return query

    
get_partition_query("", "company_id")

'updatesTable.id = targetTable.id'

In [13]:
from pyspark.sql.types import StructType, StructField, IntegerType, LongType, StringType, FloatType
from delta.tables import *
from pyspark.sql import DataFrame
  
def getTableColumn(my_schema):
    tableList = []
    col = []
    inputTable = "updatesTable."
    targetTable = "targetTable."

    ignoreColumns = ["table_key_ts", "table_delete", "table_key"]

    for field in my_schema.fields:
        if field.name == "after":
            for f3 in field.dataType:
                te = f3.name + " " + f3.dataType.typeName()
                if f3.name not in ignoreColumns:
                    tableList.append(te)
                    col.append(f3.name)
        else:
            te = field.name + " " + field.dataType.typeName()
            if field.name not in ignoreColumns:
                tableList.append(te)
                col.append(field.name)
    res = ""
    insertResInp = ""
    insertResOut = ""
    merge = ""
    for i in range(len(tableList)):
        if i == (len(tableList) - 1):
            res += tableList[i]
            insertResInp += col[i]
            insertResOut += inputTable + col[i]
            merge += targetTable + col[i] + " = " + inputTable + col[i]
        else:
            res += tableList[i] + ","
            insertResInp += col[i] + ","
            insertResOut += inputTable + col[i] + ","
            merge += targetTable + col[i] + " = " + inputTable + col[i] + ","
    ans = [res, insertResInp, insertResOut, merge]
    
    
    print(ans)
    return ans


class KafkaJob:

    def __init__(self, record, global_variables):
        self.test_data = global_variables.test_data
        self.test_schema = global_variables.test_schema
        self.sc = global_variables.sc
        self.spark = global_variables.spark
        self.schemaRegistryAddress = global_variables.schemaRegistryUrl
        self.database = record["database"]
#         self.batchOutputPath = global_variables.batchOutputPath
        
        #change 1
        self.batchOutputPath = record["batchOutputPath"]
        
#         self.kafkaGrp = global_variables.kafka_grp
        
#         #change 2
#         self.kafkaGrp = record["kafka_grp"]
        
        
#         self.OutputPathForMaterializedView = global_variables.OutputPathForMaterializedView
        
        # change 3
        self.OutputPathForMaterializedView = record["OutputPathForMaterializedView"]
        
        self.schemasByIdDict = {}
        self.bootstrapServers = global_variables.bootstrapServers
        
        # change 4
#         self.kafkaConsumerGroupForLag = record["kafkaConsumerGroupForLag"]
        
        self.kafkaConsumerGroupForLag = global_variables.kafkaConsumerGroupForLag
        
        
        self.startDateOfPipeline = global_variables.startDateOfPipeline
        self.timestampToStartStreaming = global_variables.timestampToStartStreaming
        self.table = record["table_name"]
        
        self.secondary_key = record["secondary_key"]
        self.topic_prefix = record["topic_prefix"]
        self.partition_column = record["partition_column"]
        self.zorder_column = record["zorder_column"]
        self.partition_column_derive_from_column = record["partition_column_derive_from_column"]
        self.primary_key = record["primary_key"]
        self.reference_index = record["reference_index"]
        self.topic_pattern_to_subscribe = record["topic_pattern"]
        self.is_pattern = record["is_pattern"]
        self.batchOutputPathForTopic = "{}{}/{}".format(self.batchOutputPath, self.database, self.table)
        self.checkpoint = self.batchOutputPathForTopic + "_AvroCheckpoint"
        self.OutputPathForMaterializedViewPath = "{}{}/{}".format(self.OutputPathForMaterializedView, self.database, self.table)
        self.table_exists = False
        self.subriction_option = "subscribePattern"
        self.current_primary_key = self.sc.broadcast(self.primary_key)
        self.current_secondary_key = self.sc.broadcast(self.secondary_key)
        

        self.spark.sql("create database  IF NOT EXISTS  {} LOCATION  '{}'".format(self.database, self.OutputPathForMaterializedView))
        self.spark.sql("use {}".format(self.database))
        print(self.table)
        self.new_partition_key = None
        self.new_partition_value = None
        self.byteArrayToLong = fn.udf(lambda x: int.from_bytes(x, byteorder='big', signed=False), LongType())
        self.schemasByIdDict = {}
#         val = sc._jvm.example13.Hello.add(self.bootstrapServers, self.kafkaConsumerGroupForLag)


        scalaDfLag = self.sc._jvm.com.example.Hello.add(self.spark._jsparkSession, dbutils.widgets.get("bootstrapServers"), dbutils.widgets.get("kafkaConsumerGroupForLag"))

        print(scalaDfLag)
#         print("query")
#         print(val)
        
        if self.partition_column_derive_from_column:
            partition_column_derive_from_column_split = self.partition_column_derive_from_column.split(",")
            self.new_partition_key = self.sc.broadcast(partition_column_derive_from_column_split[0])
            self.new_partition_value = self.sc.broadcast(partition_column_derive_from_column_split[1])

        if not self.is_pattern:
            self.subriction_option = "subscribe"
            
            
        self.spark.sql("SET spark.databricks.delta.schema.autoMerge.enabled = true")
        
        
        # adding option of lowshuffle merge but will work with DBR9.0+ so uncomment below as needed
#         self.spark.sql("SET spark.databricks.delta.merge.enableLowShuffle = true")
        
        self.decode_process_data = fn.udf(KafkaJob.decode_process_data_udf, StringType())

    @staticmethod
    def decode_process_data_udf(coded_string):
        result = ""
        try:
            temp = base64.b64decode(coded_string)
            result = gzip.decompress(temp).decode('utf-8')
        except:
            result = coded_string
        return result

    @staticmethod
    def get_multiple_partition_query(partition_column: str, primary_key: str):

        if not primary_key or not partition_column:
            return "updatesTable.id = targetTable.id"
        
#         query = "updatesTable.{0} = targetTable.{0}".format(primary_key)
        
        query = ""
        
        primary_keyA = []
    
        pkey_array = primary_key.split(",")
        for partitions in pkey_array:
            partitions = partitions.strip()
            if query == "":
              query += "updatesTable.{0} = targetTable.{0}".format(partitions)
            else:
              query += " and updatesTable.{0} = targetTable.{0}".format(partitions)
              
            primary_keyA.append(partitions)
        
#         primary_key = primary_key.strip()
        partitions_array = partition_column.split(",")
        for partitions in partitions_array:
            partitions = partitions.strip()
            if partitions not in primary_keyA:
                query += " and updatesTable.{0} = targetTable.{0}".format(partitions)
        return query
    
    
    @staticmethod
    def get_secondary_partition_query(partition_column: str, primary_key: str, secondary_key: str):
        
        query = "updatesTable.{0} = targetTable.{0}".format(primary_key)
        
        query += " and updatesTable.{0} = targetTable.{0}".format(secondary_key)
        
        primary_key = primary_key.strip()
        partitions_array = partition_column.split(",")
        for partitions in partitions_array:
            partitions = partitions.strip()
            if partitions != primary_key and partitions != secondary_key:
                query += " and updatesTable.{0} = targetTable.{0}".format(partitions)
        return query

      
      
    @staticmethod
    def get_partition_query(partition_column: str, primary_key: str):

        if not primary_key or not partition_column:
            return "updatesTable.id = targetTable.id"
        query = "updatesTable.{0} = targetTable.{0}".format(primary_key)
        primary_key = primary_key.strip()
        partitions_array = partition_column.split(",")
        for partitions in partitions_array:
            partitions = partitions.strip()
            if partitions != primary_key:
                query += " and updatesTable.{0} = targetTable.{0}".format(partitions)
        return query

    def getJsonSchema(self, schemaId, schemasById, registrySchemaUrl):
        jsonSchema = schemasById.get(schemaId)
        if jsonSchema is None:
            src = SchemaRegistryClient(registrySchemaUrl)
            schema = src.get_by_id(schemaId)
            jsonSchema = json.dumps(schema.flat_schema)
            schemasById[schemaId] = jsonSchema
        return jsonSchema

    @staticmethod
    def generator(partition):
      """
      Function yielding some result created by some function applied to each row of a partition (in this case lower-casing a string)

      @partition: iterator-object of partition
      """

      partCount = 1
      for row in partition:
          yield [row["offset"].lower(), row, partCount]
          partCount+=1
        
    @staticmethod
    def groupk(partition):
      from itertools import groupby
      
      partCount = 1
      l = []
      for key, group in groupby(sorted(partition, key = lambda x: x["table_key"]), lambda x: x["table_key"]):
        a = []
        k = None
        temp = -1

        for thing in group:
          if temp < int(thing["offset"]):
            temp = thing["offset"]
            k = thing

#         yield k
        l.append(k)
        partCount += 1
    
      return l
    
    def upsertToDelta(self,microBatchOutputDF, batchId):
        microBatchOutputDF = microBatchOutputDF.drop("valuenew")
        
        microBatchOutputDF = microBatchOutputDF.withColumn('valueConsumerKafka', microBatchOutputDF['value'])
        microBatchOutputDF = microBatchOutputDF.drop("value")
        
        
        microBatchOutputDF = microBatchOutputDF.withColumn("table_delete", F.when(
            col("valueConsumerKafka.before").isNotNull and col("valueConsumerKafka.op").isNotNull and col("valueConsumerKafka.op").contains("d"),
            True).otherwise(False))
        
        
        if self.secondary_key:
          microBatchOutputDF = microBatchOutputDF.withColumn("table_key", F.when(col("table_delete")==True, 
                                                                                 F.concat(microBatchOutputDF[
                                                                                     "valueConsumerKafka.before.{}".format(self.current_primary_key.value)],
                                                                                          F.lit('_'), 
                                                                                          microBatchOutputDF["valueConsumerKafka.before.{}".format(
                                                                                         self.current_secondary_key.value)])).otherwise(F.concat(microBatchOutputDF[
                                                                                     "valueConsumerKafka.after.{}".format(self.current_primary_key.value)],F.lit('_'), 
                                                                                         microBatchOutputDF["valueConsumerKafka.after.{}".format(
                                                                                         self.current_secondary_key.value)])))
        else:
          microBatchOutputDF = microBatchOutputDF.withColumn("table_key", F.when(col("table_delete") == True,
                                                                                 microBatchOutputDF[
                                                                                     "valueConsumerKafka.before.{}".format(
                                                                                         self.current_primary_key.value)]).otherwise(
              microBatchOutputDF["valueConsumerKafka.after.{}".format(self.current_primary_key.value)]))

        
          
        
        print("starting point")
        #logger.debug("starting point")

        print(datetime.datetime.now())
        
        #logger.debug(str(datetime.datetime.now()))
        
#         microBatchOutputDF.show()
        # display(microBatchOutputDF)
        print("total partitions are")
        print(microBatchOutputDF.rdd.getNumPartitions())
      
      
#         dfByTopic = microBatchOutputDF.withColumn("rn", F.row_number().over(
#             Window.partitionBy("table_key").orderBy(F.col("offset").desc())))
#         dfByTopic = dfByTopic.filter(F.col("rn") == 1).drop("rn")
      
      
#         dfByTopic2 = microBatchOutputDF.withColumn("max_offset", F.max("offset").over(
#             Window.partitionBy("table_key")))
        
#         dfByTopic = dfByTopic2.where(F.col("offset") == F.col("max_offset")).drop("max_offset")
    
        
#         # option 1 [still it requires group by but broadcasts it to multiple so maybe less shuffles]
#         dfMax = microBatchOutputDF.groupBy(col("table_key").alias("max_table_key_con")).agg(sparkMax(col("offset")).alias("max_offset_value_con"))

#         dfTopByJoin = microBatchOutputDF.join(broadcast(dfMax),
#         (col("table_key") == col("max_table_key_con")) & (col("offset") == col("max_offset_value_con"))).drop("max_table_key_con").drop("max_offset_value_con")
        
# #         for same offset can it have same key value??
#         dfByTopic = dfTopByJoin.dropDuplicates(["table_key","offset"])
  
# #         dfByTopic = spark.createDataFrame(dfByTopic2.rdd, schema = microBatchOutputDF.schema)

# #         # option 1
        
        
#         #option 2 [reducebyKey is also wide operation but it have less shuffles as can combine first and then shuffles will be less]

#         dfrdd = microBatchOutputDF.rdd.map(lambda x: (x.table_key, x)).reduceByKey(lambda x, y: x if (x.offset > y.offset) else y)

#         a = microBatchOutputDF.schema
  
#         schema_str = StructType([
#           StructField('table_key', IntegerType(), True),
#           StructField('others', a)])
    
#         dfByT = spark.createDataFrame(dfrdd, schema = schema_str)
      
#         dfByTopic = dfByT.select("others.*")
  
#         # option 2


        dfrdd = microBatchOutputDF.rdd.mapPartitions(KafkaJob.groupk)
        dfByTopic = spark.createDataFrame(dfrdd, schema = microBatchOutputDF.schema)
    
#         dfByTopic = microBatchOutputDF

#         dfByTopic = microBatchOutputDF
        print(dfByTopic.rdd.getNumPartitions())
        #logger.debug("ending point")
        
        print(datetime.datetime.now())
        
        #logger.debug(str(datetime.datetime.now()))
        dfByTopic.show()
        display(dfByTopic)

        dfByTopicDeleted = dfByTopic.filter(F.col("table_delete") == True)
        dfByTopicDeleted = dfByTopicDeleted.select("valueConsumerKafka.before.*", "table_delete", "table_key", "offset")

        dfByTopicDeleted = dfByTopicDeleted.drop("valueConsumerKafka", "kafka-key", "schemaId")
        dfByTopicDeleted = dfByTopicDeleted.drop("offset")

        dfByTopicInsert = dfByTopic.filter(F.col("table_delete") == False)
        dfByTopicInsert = dfByTopicInsert.select("valueConsumerKafka.after.*", "table_delete", "table_key", "offset")
        # display(df)
        dfByTopicInsert = dfByTopicInsert.drop("valueConsumerKafka", "kafka-key", "schemaId")
        dfByTopicInsert = dfByTopicInsert.drop("offset")
        if self.partition_column_derive_from_column:
            dfByTopicDeleted = dfByTopicDeleted.withColumn(self.new_partition_key.value,
                                                           to_date(col(self.new_partition_value.value)))
            dfByTopicInsert = dfByTopicInsert.withColumn(self.new_partition_key.value,
                                                         to_date(col(self.new_partition_value.value)))
        if self.table == 'process':
            dfByTopicInsert = dfByTopicInsert.withColumn("process_data", self.decode_process_data("process_data"))

        if not self.table_exists:
            my_schema = dfByTopicInsert.schema
            res = getTableColumn(my_schema)
            dfByTopic._jdf.sparkSession().sql("use {}".format(self.database))
            if self.partition_column:
                dfByTopic._jdf.sparkSession().sql(
                    "create table if not exists {} ( {} ) using delta PARTITIONED BY ({}) location '{}' ".format(self.table,
                                                                                                                 res[0],
                                                                                                                 self.partition_column,
                                                                                                                 self.OutputPathForMaterializedViewPath))
            else:
                dfByTopic._jdf.sparkSession().sql(
                    "create table if not exists {} ( {} ) using delta location '{}' ".format(self.table, res[0],
                                                                                             self.OutputPathForMaterializedViewPath))

            self.table_exists = True

        dfByTopicInsert = dfByTopicInsert.drop("table_delete")
        dfByTopicInsert = dfByTopicInsert.drop("table_key")

#         PYSPARK MERGE
#         deltaTable=DeltaTable.forName(spark,self.table)
        FullTableName = self.database + "." + self.table

        print(FullTableName)

#         deltaTable = DeltaTable.forName(spark, FullTableName)
        
        deltaTable=DeltaTable.forName(spark,FullTableName)
    
#         jab
        
        if self.secondary_key:
            if not self.partition_column:
              query = KafkaJob.get_secondary_partition_query("", self.primary_key, self.secondary_key)
            else:
              query = KafkaJob.get_secondary_partition_query(self.partition_column, self.primary_key, self.secondary_key)

            deltaTable.alias("targetTable").merge(
                dfByTopicDeleted.alias("updatesTable"),
                "{0}".format(query)) \
              .whenMatchedDelete() \
              .execute()


            deltaTable.alias("targetTable").merge(
                dfByTopicInsert.alias("updatesTable"),
                "{0}".format(query)) \
              .whenMatchedUpdateAll() \
              .whenNotMatchedInsertAll().execute()

        elif self.partition_column and self.partition_column != self.primary_key:
            query = KafkaJob.get_partition_query(self.partition_column, self.primary_key)

            deltaTable.alias("targetTable").merge(
                dfByTopicDeleted.alias("updatesTable"),
                "{0}".format(query)) \
              .whenMatchedDelete() \
              .execute()


            deltaTable.alias("targetTable").merge(
                dfByTopicInsert.alias("updatesTable"),
                "{0}".format(query)) \
              .whenMatchedUpdateAll() \
              .whenNotMatchedInsertAll().execute()

        else:
            
            deltaTable.alias("targetTable").merge(
                dfByTopicDeleted.alias("updatesD"),
                "updatesD.table_key = targetTable.{0}".format(self.primary_key)) \
              .whenMatchedDelete() \
              .execute()

            deltaTable.alias("targetTable").merge(
                dfByTopicInsert.alias("updatesI"),
                "updatesI.{0} = targetTable.{0}".format(self.primary_key)) \
              .whenMatchedUpdateAll() \
              .whenNotMatchedInsertAll().execute()

#         PYSPARK MERGE

        print("after merge")

        #logger.debug("after merge")
        
        print(datetime.datetime.now())
        
        #logger.debug(str(datetime.datetime.now()))

    def get_avro_data_by_schema_for_each(self,df, batchId):
      
        self.reference_index = batchId % 50
        print("optimize check")
        
        ldf = spark.sql("select (int(current_timestamp()) - int(last_updated))/60 from pipeline_common.vacuumStatus where table = '" + self.database + "." + self.table + "'")
        
        if ldf.first()[0] > 5:
          if batchId % 50 == int(self.reference_index) and batchId >= 50:
              try:
                  print("optimizing {}.{}".format(self.database,self.table))
                  if self.is_pattern:
                    print("using partition last 3 month")

                    timeCol = self.partition_column_derive_from_column.split(",")[0] 

                    if self.zorder_column and batchId % 150 == int(self.reference_index):
                        self.spark.sql("optimize {0}.{1} WHERE {3} >= current_timestamp() - INTERVAL 100 day ZORDER BY ({2})".format(self.database, self.table, self.zorder_column, timeCol))
                    else:
                        self.spark.sql("optimize {0}.{1} WHERE {2} >= current_timestamp() - INTERVAL 100 day ".format(self.database, self.table, timeCol))
                  else:
                    if self.zorder_column and batchId % 150 == int(self.reference_index):
                        self.spark.sql("optimize {0}.{1} ZORDER BY ({2})".format(self.database, self.table, self.zorder_column))
                    else:
                        self.spark.sql("optimize {0}.{1}".format(self.database, self.table))

                  print("vacuum {}.{}".format(self.database,self.table))
                  self.spark.sql("vacuum {0}.{1}".format(self.database, self.table))
                  spark.sql("update pipeline_common.vacuumStatus set last_updated = current_timestamp() where table = '" + self.database + "." + self.table + "'")
        
              except Exception as e:
                  print(e)
        month_year = ""
        avro_topic = self.topic_pattern_to_subscribe + "-value"
        if self.is_pattern:
            month_year = "_" + str(_datetime.date.today().month) + "_" + str(_datetime.date.today().year)
            avro_topic = self.topic_prefix + self.table + month_year + "-value"
        print("avro_topic: " + avro_topic)
        avro_topic_br = self.sc.broadcast(avro_topic)
        try:
            print("running fast: {}", avro_topic)
            df = df.withColumn('valuenew', df['value'])
            df = df.withColumn('value', from_avro("value", avro_topic_br.value, self.schemaRegistryAddress))
            
            self.upsertToDelta(df, batchId)
        except Exception as fastException:
#           print("checking aa")  
            try:
                mem_limit = 2 ** 20
                print("running slow: {}", avro_topic)
                print("fastException try slow", fastException)
                if sys.getsizeof(self.schemasByIdDict) > mem_limit:
                    self.schemasByIdDict.clear()
                df = df \
                    .withColumn("schemaId", self.byteArrayToLong(fn.substring("valuenew", 2, 4))) \
                    .withColumn("payload", fn.expr("substring(valuenew, 6, length(valuenew)-5)"))
                dfAllSchemas = df.select("schemaId").distinct()
                schemaRowList = dfAllSchemas.collect()
                print(schemaRowList)
                for schemaRow in schemaRowList:
                    jsonSchema = self.getJsonSchema(schemaRow.schemaId, self.schemasByIdDict,
                                                    'http://schema-registry:8081')
                    currentValueSchemaId = self.sc.broadcast(schemaRow.schemaId)
                    currentValueSchema = self.sc.broadcast(jsonSchema)
                    dfBySchemaId = df.where(df.schemaId == currentValueSchemaId.value)
                    dfBySchemaId = dfBySchemaId.drop("value")
                    dfBySchemaId = dfBySchemaId.withColumn("value", from_avro("payload", currentValueSchema.value))
                    dfBySchemaId = dfBySchemaId.drop("payload")
                    self.upsertToDelta(dfBySchemaId, batchId)
            except Exception as slowException:
                print("slowException {}".format(avro_topic), fastException)
                raise slowException

    def start_read_stream(self):
      
        print("starting stream")
        #logger.debug("starting stream")

        print(datetime.datetime.now())
        
        #logger.debug(str(datetime.datetime.now()))
        
        print("topic_pattern_to_subscribe: " + self.topic_pattern_to_subscribe)
        
        
#         GrpId = "fareye_" + self.kafkaGrp + "_" + self.topic_pattern_to_subscribe
        kafka = self.spark.readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", self.bootstrapServers) \
            .option(self.subriction_option, self.topic_pattern_to_subscribe) \
            .option("startingOffsets", "earliest") \
            .option("maxOffsetsPerTrigger", 1000000) \
            .option("failOnDataLoss", False) \
            .load()

        if self.timestampToStartStreaming is not None:
            kafka = kafka.filter(kafka['timestamp'] >= self.timestampToStartStreaming)

        kafka = kafka.filter("value is NOT NULL")
        return kafka

    def start_write_stream(self,kafka,method_to_run_on_for_each):
      
        print("starting write")
        #logger.debug("starting write")

        print(datetime.datetime.now())
        
        #logger.debug(str(datetime.datetime.now()))
        
        ssc = kafka \
            .writeStream \
            .queryName("{0}_{1}".format(self.database,self.table)) \
            .option("checkpointLocation", self.checkpoint) \
            .trigger(processingTime='60 seconds') \
            .foreachBatch(method_to_run_on_for_each).start()
        ssc.awaitTermination()


    def start_pipeline_with_avro(self):
        kafka = self.start_read_stream()
        self.start_write_stream(kafka,self.get_avro_data_by_schema_for_each)

    def start_pipeline_without_avro(self):
        kafka = self.start_read_stream()
        self.start_write_stream(kafka,self.upsertToDelta)

In [ ]:
        
#         # option 1 [still it requires group by but broadcasts it to multiple so maybe less shuffles]
#         dfMax = microBatchOutputDF.groupBy(col("table_key").alias("max_table_key")).agg(sparkMax(col("offset")).alias("max_offset_value"))

#         dfByTopic = microBatchOutputDF.join(broadcast(dfMax),
#         (col("table_key") == col("max_table_key")) & (col("offset") == col("max_offset_value"))).drop("max_table_key").drop("max_offset_value")
        
#         # for same offset can it have same key value??
#         #  dfByTopic = dfTopByJoin.groupBy("table_key").agg(
#         #  first("category").alias("category"),
#         #  first("TotalValue").alias("TotalValue"))

#         # option 1
        
        
#         #option 2 [reducebyKey is also wide operation but it have less shuffles as can combine first and then shuffles will be less]

#         print("option 2 debug")
#         dfrdd = microBatchOutputDF.rdd.map(lambda x: (x.table_key, x)).reduceByKey(lambda x, y: x if (x.offset > y.offset) else y)

#         a = microBatchOutputDF.schema
  
#         schema_str = StructType([
#           StructField('table_key', IntegerType(), True),
#           StructField('others', a)])
    
#         dfByT = spark.createDataFrame(dfrdd, schema = schema_str)
      
#         dfByTopic = dfByT.select("others.*")
  
#         # option 2

        
  
        # identified as best among others
        # option 3 using foreach partition (which is based on condition that a particular table key which always be in same partition and not in any other which is taken care by debziumn and kafka)

#         def f(a):
#             print(a)
        
#         res = microBatchOutputDF.rdd.mapPartitions(KafkaJob.groupk).take(5) #toDF().select("_2.*")
        
#         [f(e) for e in res]


In [ ]:
class Global_variables():
    pass

In [ ]:
from pyspark.sql.avro.functions import from_avro, to_avro
from pyspark.sql.functions import col
from pyspark.sql.functions import lit
from pyspark.sql import functions as F
from pyspark.sql.types import StructType
from pyspark.sql.functions import expr
from pyspark.sql.types import StringType
from pyspark.sql.window import Window
from pyspark.sql.functions import to_timestamp
from pyspark import Row
import random
from pyspark.sql.types import LongType, StringType
import pyspark.sql.functions as fn
from pyspark.sql.types import *
from pyspark.sql.functions import to_date
from pyspark.sql.functions import to_json,from_json,schema_of_json
import sys
from schema_registry.client import SchemaRegistryClient, schema
import json 
from pyspark.sql.functions import udf
import base64
import gzip 
import _datetime
import datetime
import time
import requests

# from pyspark.sql import SparkSession
# from databricks_pipeline.Dbutils import get_dbutils
# from databricks_pipeline.Global_variables import Global_variables
# from schema_registry.client import SchemaRegistryClient
# import json
# import datetime
# from databricks_pipeline import pipeline

In [ ]:
spark = SparkSession.builder.config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
            .getOrCreate()

In [ ]:
# spark.sparkContext.setLogLevel("DEBUG")
# Logger= spark._jvm.org.apache.log4j.Logger
# mylogger = Logger.getLogger(__name__)

# print(mylogger)

In [ ]:
%sh
#!/bin/bash
echo "Adjusting log4j.properties here: ${LOG4J_PATH}"

Adjusting log4j.properties here:

In [ ]:
# mylogger.debug("debug message")

In [ ]:
%scala
// sc.parallelize(Seq("")).foreachPartition(x => {
//   import org.apache.log4j.{LogManager, Level}
//   import org.apache.commons.logging.LogFactory

//   LogManager.getRootLogger().setLevel(Level.ERROR)
//   val log = LogFactory.getLog("EXECUTOR-LOG:")
//   log.debug("START EXECUTOR ERROR LOG LEVEL")
// })

In [ ]:
schemaRegistryAddress = None
batchOutputPath = None
kafkaGrp = None
OutputPathForMaterializedView = None
schemasByIdDict = {}
bootstrapServers = None
kafkaConsumerGroupForLag = None
startDateOfPipeline = None
schemaRegistryAddress = dbutils.widgets.get("schemaRegistryUrl")
# batchOutputPath = dbutils.widgets.get("batchOutputPath")
# kafkaGrp = dbutils.widgets.get("kafka-grp")
job_database = dbutils.widgets.get("database")
# OutputPathForMaterializedView = dbutils.widgets.get("OutputPathForMaterializedView")
schemasByIdDict = {}
bootstrapServers = dbutils.widgets.get("bootstrapServers")
# kafkaConsumerGroupForLag = dbutils.widgets.get("kafkaConsumerGroupForLag")
startDateOfPipeline = dbutils.widgets.get("startDateOfPipeline")
# TO_EMAIL = dbutils.widgets.get("TO_EMAIL")
# EMAIL_SERVICE_URL = dbutils.widgets.get("EMAIL_SERVICE_URL")
# EMAIL_SERVICE_API_KEY = dbutils.widgets.get("EMAIL_SERVICE_API_KEY")
# SUBJECT_EMAIL = dbutils.widgets.get("SUBJECT_EMAIL")
job_database = dbutils.widgets.get("database")

# print("kafkaGrp: " + kafkaGrp)
# print("bootstrapServers: " + bootstrapServers)
# print("schemaRegistryUrl: " + schemaRegistryAddress)
# print("kafkaConsumerGroupForLag: " + kafkaConsumerGroupForLag)
# print("startDateOfPipeline : " + startDateOfPipeline)
timestampToStartStreaming = None

if startDateOfPipeline != "":
    timestampToStartStreaming = datetime.datetime.strptime(startDateOfPipeline, "%d-%m-%Y")
    print("timestamp: " + str(timestampToStartStreaming))

timestamp: 2021-11-05 00:00:00

In [ ]:
tabledf = spark.sql("select * from pipeline_common.pipeline_tables where database_name in ({0})".format(job_database))
display(tabledf)

database_name,table_name,partition_column,zorder_column,partition_column_derive_from_column,primary_key,reference_index,topic_pattern,table_exists,is_pattern,topic_prefix,kafkaConsumerGroupForLag,OutputPathForMaterializedView,kafka_grp,batchOutputPath,secondary_key
dev_pipeline_test_optimize,runsheet,"company_id,start_date_part",id,"start_date_part,start_date",id,0,qatest12nonfes.public.runsheet_old,false,false,,databricks-datalake,/mnt/fareye-datalake-ca-test/processed55/,datalake2,/mnt/fareye-datalake-ca-test/raw55/,null
dev_pipeline_test_optimize,user_summary_old,"company_id,date",id,null,id,1,qatest12nonfes.public.user_summary_old,false,false,,databricks-datalake,/mnt/fareye-datalake-ca-test/processed55/,datalake2,/mnt/fareye-datalake-ca-test/raw55/,null


In [ ]:
# COMMAND ----------

def toMap(record):
    data = {};
    data["partition_column"] = record["partition_column"];
    data["zorder_column"] = record["zorder_column"];
    data["primary_key"] = record["primary_key"];
    data["partition_column_derive_from_column"] = record["partition_column_derive_from_column"];
    data["table_name"] = record["table_name"];
    data["reference_index"] = record["reference_index"];
    data["topic_pattern"] = record["topic_pattern"];
    data["table_exists"] = record["table_exists"];
    data["is_pattern"] = record["is_pattern"];
    data["topic_prefix"] = record["topic_prefix"];
    data["database"] = record["database_name"];
    
    data["OutputPathForMaterializedView"] = record["OutputPathForMaterializedView"];
    data["kafkaConsumerGroupForLag"] = record["kafkaConsumerGroupForLag"];
    data["kafka_grp"] = record["kafka_grp"];
    data["batchOutputPath"] = record["batchOutputPath"];
    
    

    return (record['table_name'], data)

In [ ]:
# COMMAND ----------

mapOfTopicAndData = tabledf.rdd.map(lambda record: toMap(record)).collectAsMap()
mapForTableToPartition = spark.sparkContext.broadcast(mapOfTopicAndData)

In [ ]:
def getPrefixTopic(topicToProcess:str):
  if '_old' in topicToProcess:
    return topicToProcess.replace("_old","")
  a = topicToProcess.split("_")
  try:
    month = int(a[len(a)-2])
    year = int(a[len(a)-1])
    if month>=1 and month <=12 and year >=2019:
      res = ""
      for i in range(len(a)-2):
         if i == (len(a)-3): 
             res+=a[i]
         else:
             res+=a[i]+"_"
      return res    
  except:        
    pass

  return topicToProcess

In [ ]:
global_variables = Global_variables()
global_variables.sc = spark.sparkContext
global_variables.spark = spark
global_variables.schemaRegistryUrl = schemaRegistryAddress
# global_variables.batchOutputPath = batchOutputPath
# global_variables.kafka_grp = kafkaGrp
# global_variables.OutputPathForMaterializedView = OutputPathForMaterializedView
global_variables.bootstrapServers = bootstrapServers
# global_variables.kafkaConsumerGroupForLag = kafkaConsumerGroupForLag
global_variables.startDateOfPipeline = startDateOfPipeline
global_variables.timestampToStartStreaming = timestampToStartStreaming
global_variables.test_data = None
global_variables.test_schema = None

In [ ]:
# %python
# from pyspark.sql import DataFrame

# df4 = sc._jvm.example10.Hello.add(dbutils.widgets.get("bootstrapServers"), dbutils.widgets.get("kafkaConsumerGroupForLag"))

# print(df4)

In [ ]:
 %scala
//  import java.util
//  import com.fasterxml.jackson.databind.{DeserializationFeature, ObjectMapper}
//  import com.fasterxml.jackson.module.scala.DefaultScalaModule
//  import org.apache.kafka.clients.consumer.{KafkaConsumer, OffsetAndMetadata}
//  import org.apache.kafka.common.TopicPartition
//  import org.apache.kafka.clients.KafkaClient
//  import java.util.Properties
//  import org.apache.spark.sql.streaming.StreamingQueryListener
//  import org.apache.spark.sql.streaming.StreamingQueryListener._
 
//  spark.streams.addListener(new StreamingQueryListener() {
 
//      val props = new Properties()
//      val bootstrapServers =  dbutils.widgets.get("bootstrapServers")
//      val kafkaConsumerGroupForLag  =  dbutils.widgets.get("kafkaConsumerGroupForLag")
//      props.put("bootstrap.servers", bootstrapServers)
//      props.put("group.id", kafkaConsumerGroupForLag )
//      props.put("key.deserializer", "org.apache.kafka.common.serialization.StringDeserializer")
//      props.put("value.deserializer", "org.apache.kafka.common.serialization.StringDeserializer")
//      private val consumer = new KafkaConsumer[String, String](props)
//      override def onQueryStarted(queryStarted: QueryStartedEvent): Unit = {
//          println("Query started: " + queryStarted.id)
//      }
//      override def onQueryTerminated(queryTerminated: QueryTerminatedEvent): Unit = {
//          println("Query terminated: " + queryTerminated.id)
//      }
//      override def onQueryProgress(event: QueryProgressEvent): Unit = {
 
//        val om = new ObjectMapper().configure(DeserializationFeature.FAIL_ON_UNKNOWN_PROPERTIES, false)
//        om.registerModule(DefaultScalaModule)
//        event.progress.sources.foreach(source => {
//          val jsonOffsets = om.readValue(source.endOffset, classOf[Map[String, Map[String, Int]]])
//          jsonOffsets.keys
//            .foreach(topic => {
//              val topicPartitionMap = new util.HashMap[TopicPartition, OffsetAndMetadata]()
//              val offsets = jsonOffsets.get(topic)
//              offsets match {
//                case Some(topicOffsetData) =>
//                  topicOffsetData.keys.foreach(partition => {
//                    val tp = new TopicPartition(topic, partition.toInt)
//                    val oam = new OffsetAndMetadata(topicOffsetData(partition).toLong)
//                    topicPartitionMap.put(tp, oam)
//                  })
//                case _ =>
 
//              }
//              consumer.commitSync(topicPartitionMap)
//              println("Query made progress: " + topicPartitionMap)
//            })
//        })
//      }
//  })

In [ ]:
def getJsonSchema(schemaId, schemasById, registrySchemaUrl):
  jsonSchema = schemasById.get(schemaId)
  if jsonSchema is None:
    src = SchemaRegistryClient(registrySchemaUrl)
    schema = src.get_by_id(schemaId)
    jsonSchema = json.dumps(schema.flat_schema)
    schemasById[schemaId] = jsonSchema
  return jsonSchema


In [ ]:
def getTableColumn(my_schema):
  tableList = []
  col = []
  inputTable = "updatesTable."
  targetTable = "targetTable."
  
  ignoreColumns = ["table_key_ts","table_delete","table_key"]
 
  for field in my_schema.fields:
    if field.name == "after":
        for f3 in field.dataType:
          te = f3.name+" "+f3.dataType.typeName()
          if f3.name not in ignoreColumns:
             tableList.append(te) 
             col.append(f3.name) 
    else:
        te = field.name+" "+field.dataType.typeName()
        if field.name not in ignoreColumns:
          tableList.append(te)
          col.append(field.name)
  res = ""
  insertResInp = ""
  insertResOut = ""
  merge = ""
  for i in range(len(tableList)): 
      if i == (len(tableList)-1): 
         res+=tableList[i]
         insertResInp+=col[i]
         insertResOut+=inputTable+col[i]
         merge+=targetTable+col[i] + " = " + inputTable+col[i]         
      else:
         res+=tableList[i]+","
         insertResInp+=col[i]+","          
         insertResOut+=inputTable+col[i]+","
         merge+=targetTable+col[i]+ " = " + inputTable+col[i] +","
  ans = []
  ans.append(res)
  ans.append(insertResInp)
  ans.append(insertResOut)
  ans.append(merge)
#   print(ans)
  return ans  
       

In [ ]:
# # deltaStream = spark.readStream.format("delta").load("/tmp/fareye-datalake/prod_pipeline_xx_1/track_log")
# # display(deltaStream)

# def send_alert_mail(topic):
#   mailData = '{{"fromEmail":"no-reply@getfareye.com", "toEmail":"saurabh.nigam@getfareye.com", "sub":"Alert in pipeline aws ca","messageBody":"Pipeline stopped topic: {}","serverName":"databricks-aws-ca"}}'.format(topic)
#   return
#   headers = {'apiKey': 'auX6DIJk4ZAF2Q1505373210570ung72CmxZIsrnLM27K'}
#   payload = {
#       'mailData': (None, mailData,'application/json')
#   }
#   response = requests.post('https://email-service.fareye.co/app/rest/mail/sendMail', files=payload, headers=headers)
#   print(response.json())

In [ ]:
# def kafka_job_wrapper(record):
#     try:
#         kafka_job(record)
#     except Exception as e:
#         print("Exception in wrapper.{0}".format(record["table_name"]), e)
# #         send_alert_mail(record["table_name"])

In [ ]:
# from concurrent.futures import ThreadPoolExecutor

# def parallelKafkaWorkers(dataMap, numInParallel):
#     returnRes = []
#     with ThreadPoolExecutor(max_workers=100) as ec:
#         for index in dataMap:

#             if dataMap[index]:
#                 print(index)
                
#                 kj = KafkaJob(dataMap[index] , global_variables)
#                 returnRes.append(ec.submit(kafka_job_wrapper,kj))
#     return returnRes               

In [ ]:
# print(len(mapOfTopicAndData))

In [ ]:
# import  concurrent.futures
# res = parallelKafkaWorkers(mapOfTopicAndData, len(mapOfTopicAndData))
# for future in concurrent.futures.as_completed(res):
#         try:
#             print(future.result())
#         except:
#             print("Exception.")

In [ ]:
# result = [i.result() for i in res] # This is a blocking call.
# print(result)   

In [ ]:
schemaRegistryAddress = None

# batchOutputPath = None

kafkaGrp = None

# OutputPathForMaterializedView = None

schemasByIdDict = {}
bootstrapServers = None
kafkaConsumerGroupForLag = None
startDateOfPipeline = None
schemaRegistryAddress = dbutils.widgets.get("schemaRegistryUrl")

# batchOutputPath = dbutils.widgets.get("batchOutputPath")

# kafkaGrp = dbutils.widgets.get("kafka-grp")

# OutputPathForMaterializedView = dbutils.widgets.get("OutputPathForMaterializedView")

schemasByIdDict = {}
bootstrapServers = dbutils.widgets.get("bootstrapServers")
kafkaConsumerGroupForLag = dbutils.widgets.get("kafkaConsumerGroupForLag")
startDateOfPipeline = dbutils.widgets.get("startDateOfPipeline")
TO_EMAIL = dbutils.widgets.get("TO_EMAIL")
EMAIL_SERVICE_URL = dbutils.widgets.get("EMAIL_SERVICE_URL")
EMAIL_SERVICE_API_KEY = dbutils.widgets.get("EMAIL_SERVICE_API_KEY")
SUBJECT_EMAIL = dbutils.widgets.get("SUBJECT_EMAIL")
job_database = dbutils.widgets.get("database")

# print("kafkaGrp: " + kafkaGrp)
print("bootstrapServers: " + bootstrapServers)
print("schemaRegistryUrl: " + schemaRegistryAddress)
print("kafkaConsumerGroupForLag: " + kafkaConsumerGroupForLag)
print("startDateOfPipeline : " + startDateOfPipeline)
timestampToStartStreaming = None

print("db is")
print(job_database)

if startDateOfPipeline != "":
    timestampToStartStreaming = datetime.datetime.strptime(startDateOfPipeline, "%d-%m-%Y")
    print("timestamp: " + str(timestampToStartStreaming))

# COMMAND ----------


tabledf = spark.sql("select * from pipeline_common.pipeline_tables where database_name in ({0})".format(job_database))
# tabledf = spark.sql("select * from pipeline_common.pipeline_tables where database_name in ({0}) and table_name = 'field_data' ".format(job_database))

# COMMAND ----------

display(tabledf)


# COMMAND ----------

def toMap(record):
    data = {};
    data["partition_column"] = record["partition_column"];
    data["zorder_column"] = record["zorder_column"];
    data["primary_key"] = record["primary_key"];
    data["partition_column_derive_from_column"] = record["partition_column_derive_from_column"];
    data["table_name"] = record["table_name"];
    data["reference_index"] = record["reference_index"];
    data["topic_pattern"] = record["topic_pattern"];
    data["table_exists"] = record["table_exists"];
    data["is_pattern"] = record["is_pattern"];
    data["topic_prefix"] = record["topic_prefix"];
    data["database"] = record["database_name"];
    
    data["secondary_key"] = record["secondary_key"];
    
    
#     data["OutputPathForMaterializedView"] = "/mnt/fareye-datalake-ca-test/processed54/"
#     data["kafkaConsumerGroupForLag"] = "databricks-datalake"
#     data["kafka_grp"] = "datalake2"
#     data["batchOutputPath"] = "/mnt/fareye-datalake-ca-test/raw54/"
    
#     data["OutputPathForMaterializedView"] = record["OutputPathForMaterializedView"];
#     data["batchOutputPath"] = record["batchOutputPath"];
    
    data["OutputPathForMaterializedView"] = "/mnt/fareye-datalake-ca-test/processed717/";
    data["batchOutputPath"] = "/mnt/fareye-datalake-ca-test/raw717/";
    
    
    rec_comb = record['table_name'] + "_" + record['database_name']

    

    return (rec_comb, data)


# COMMAND ----------

mapOfTopicAndData = tabledf.rdd.map(lambda record: toMap(record)).collectAsMap()
mapForTableToPartition = spark.sparkContext.broadcast(mapOfTopicAndData)


# COMMAND ----------
global_variables = Global_variables()
global_variables.sc = spark.sparkContext
global_variables.spark = spark
global_variables.schemaRegistryUrl = schemaRegistryAddress

# global_variables.batchOutputPath = batchOutputPath

# global_variables.kafka_grp = kafkaGrp

# global_variables.OutputPathForMaterializedView = OutputPathForMaterializedView

global_variables.bootstrapServers = bootstrapServers
global_variables.kafkaConsumerGroupForLag = kafkaConsumerGroupForLag
global_variables.startDateOfPipeline = startDateOfPipeline
global_variables.timestampToStartStreaming = timestampToStartStreaming
global_variables.test_data = None
global_variables.test_schema = None


# COMMAND ----------


import requests

def send_alert_mail(topic):
    mailData = '{{"fromEmail":"no-reply@getfareye.com", "toEmail":"{}", "sub":"{}","messageBody":"Pipeline stopped topic: {}","serverName":"databricks"}}'.format(
        TO_EMAIL,SUBJECT_EMAIL,topic)

    headers = {'apiKey': EMAIL_SERVICE_API_KEY}
    payload = {
        'mailData': (None, mailData, 'application/json')
    }

    response = requests.post(EMAIL_SERVICE_URL, files=payload, headers=headers)
    print(response.json())

# COMMAND ----------


def getJsonSchema(schemaId, schemasById, registrySchemaUrl):
    jsonSchema = schemasById.get(schemaId)
    if jsonSchema is None:
        src = SchemaRegistryClient(registrySchemaUrl)
        schema = src.get_by_id(schemaId)
        jsonSchema = json.dumps(schema.flat_schema)
        schemasById[schemaId] = jsonSchema
    return jsonSchema


# # COMMAND ----------
def kafka_job_wrapper(kj):
    try:
        kj.start_pipeline_with_avro()
    except Exception as e:
        print("Exception in wrapper.{0}".format(kj.table), e)
        send_alert_mail(kj.table)
from concurrent.futures import ThreadPoolExecutor

def parallelKafkaWorkers(dataMap, numInParallel):
    returnRes = []
    with ThreadPoolExecutor(max_workers=100) as ec:
        for index in dataMap:

            if dataMap[index]:
                print(index)
                kj = KafkaJob(dataMap[index] , global_variables)
                returnRes.append(ec.submit(kafka_job_wrapper,kj))
    return returnRes


# COMMAND ----------
print("lenght is")
print(len(mapOfTopicAndData))

print(mapOfTopicAndData)

# COMMAND ----------

import concurrent.futures

res = parallelKafkaWorkers(mapOfTopicAndData, len(mapOfTopicAndData))
for future in concurrent.futures.as_completed(res):
    try:
        print(future.result())
    except:
        print("Exception.")

# COMMAND ----------

# try:
#    for index in mapOfTopicAndData:
#         if mapOfTopicAndData[index]:
#             print("index is")
#             print(index)
#             kj = KafkaJob(mapOfTopicAndData[index] , global_variables)
#             kj.start_pipeline_with_avro()
# except Exception as e:
#     print("Exception in wrapper.{0}".format(kj.table), e)
#     send_alert_mail(kj.table)

result = [i.result() for i in res]  # This is a blocking call.
print(result)

# COMMAND ----------

res

dev_pipeline_test_optimize.user_summary_old
after merge
2022-03-01 12:08:12.974273

In [ ]:
# %sql
# select * from pipeline_common.pipeline_tables

In [ ]:
%sql
select * from dev_pipeline_test_optimize.runsheet

In [ ]:
# %sql
# truncate table dev_pipeline_test_optimize.user_summary_old

UNIT TESTING CODE

In [ ]:
import json
from abc import ABC, abstractmethod
from argparse import ArgumentParser
from logging import Logger
from typing import Dict, Any

from pyspark.sql import SparkSession


# abstract class for jobs
class Job(ABC):
    @abstractmethod
    def init_adapter(self):
        """
        Init adapter is an abstract method to perform some particular settings in the Job subclass.
        Method is called after creation of the SparkSession.
        :return:
        """
        pass

    def __init__(self, spark=None, init_conf=None):
        self.spark = self._prepare_spark(spark)
        self.logger = self._prepare_logger()
        self.dbutils = self.get_dbutils()
        if init_conf:
            self.conf = init_conf
        else:
            self.conf = self._provide_config()
        self.init_adapter()
        self._log_conf()

    @staticmethod
    def _prepare_spark(spark) -> SparkSession:
        if not spark:
            return SparkSession.builder.getOrCreate()
        else:
            return spark

    @staticmethod
    def _get_dbutils(spark: SparkSession):
        try:
            from pyspark.dbutils import DBUtils # noqa
            if "dbutils" not in locals():
                utils = DBUtils(spark)
                return utils
            else:
                return locals().get("dbutils")
        except ImportError:
            return None

    def get_dbutils(self):
        utils = self._get_dbutils(self.spark)

        if not utils:
            self.logger.warn("No DBUtils defined in the runtime")
        else:
            self.logger.info("DBUtils class initialized")

        return utils

    def _provide_config(self):
        self.logger.info("Reading configuration from --conf-file job option")
        conf_file = self._get_conf_file()
        if not conf_file:
            self.logger.info(
                "No conf file was provided, setting configuration to empty dict."
                "Please override configuration in subclass init method"
            )
            return {}
        else:
            self.logger.info(
                f"Conf file was provided, reading configuration from {conf_file}"
            )
            return self._read_config(conf_file)

    @staticmethod
    def _get_conf_file():
        p = ArgumentParser()
        p.add_argument("--conf-file", required=False, type=str)
        namespace = p.parse_known_args()[0]
        return namespace.conf_file

    def _read_config(self, conf_file) -> Dict[str, Any]:
        raw_content = "".join(
            self.spark.read.format("text").load(conf_file).toPandas()["value"].tolist()
        )
        config = json.loads(raw_content)
        return config

    def _prepare_logger(self) -> Logger:
        log4j_logger = self.spark._jvm.org.apache.log4j # noqa
        return log4j_logger.LogManager.getLogger(self.__class__.__name__)

    def _log_conf(self):
        # log parameters
        self.logger.info("Launching job with configuration parameters:")
        for key, item in self.conf.items():
            self.logger.info("\t Parameter: %-30s with value => %-30s" % (key, item))

    @abstractmethod
    def launch(self):
        """
        Main method of the job.
        :return:
        """
        pass

In [ ]:
from pyspark.sql.functions import from_json, col


class TestJob(KafkaJob):

    def start_read_stream(self):
        rddjson = self.sc.parallelize([self.test_data])
        kafka = self.spark.read.json(rddjson)
        kafka = kafka.filter("value is NOT NULL");
        kafka= kafka.withColumn('value', from_json(col('value'), self.test_schema))
        return kafka

    def start_write_stream(self, kafka, method_to_run_on_for_each):
        self.spark.sql("create database  IF NOT EXISTS  {} LOCATION  '{}'".format(self.database, self.OutputPathForMaterializedView))
        self.spark.sql("use {}".format(self.database))
        method_to_run_on_for_each(kafka,1)

    @staticmethod
    def write_dataframe(kafka):
        kafka.write \
            .format("delta").mode("overwrite") \
            .save("/tmp/qwerty")

In [ ]:
from concurrent.futures.thread import ThreadPoolExecutor
import concurrent.futures


class PipelineJob(Job):

    def init_adapter(self):
        if not self.conf:
            self.logger.info("Init configuration was not provided, using configuration from default_init method")
            self.conf = {
                "output_format": "delta",
                "output_path": "dbfs:/dbx/tmp/test/interactive/cicd_sample_project"
            }
        else:
            self.logger.info("Init configuration is already provided")

    def launch(self):
        self.logger.info("Launching bootstrap job")

        # df = self.spark.range(0, 1000)
        # df.write.format(self.conf["output_format"]).mode("overwrite").save(self.conf["output_path"])


        self.logger.info("Bootstrap job finished!")

class TestPipelineJob(Job):

    def init_adapter(self):
        if not self.conf:
            self.logger.info("Init configuration was not provided, using configuration from default_init method")
            self.conf = {
                "output_format": "delta",
                "output_path": "dbfs:/dbx/tmp/test/interactive/cicd_sample_project"
            }
        else:
            self.logger.info("Init configuration is already provided")

    @staticmethod
    def parallelKafkaWorkers(conf, numInParallel):
        returnRes = []
        dataMap = conf.get("mapOfTopicAndData")
        global_variables = conf.get("global_variables")
        with ThreadPoolExecutor(max_workers=100) as ec:
            for index in dataMap:

                if dataMap[index]:
                    print(index)
                    kj = TestJob(dataMap[index], global_variables)
                    returnRes.append(ec.submit(kj.start_pipeline_without_avro))
        return returnRes

    def launch(self):
        self.logger.info("Launching bootstrap job")

        # df = self.spark.range(0, 1000)
        # df.write.format(self.conf["output_format"]).mode("overwrite").save(self.conf["output_path"])

        print(len(self.conf.get("mapOfTopicAndData")))
        res = self.parallelKafkaWorkers(self.conf, len(self.conf.get("mapOfTopicAndData")))
        for future in concurrent.futures.as_completed(res):
            try:
                print(future.result())
            except Exception as ex:
                print(ex)

        result = [i.result() for i in res]  # This is a blocking call.
        print(result)

        self.logger.info("Bootstrap job finished!")

if __name__ == "__main__":
    job = PipelineJob()
    job.launch()

In [ ]:
import datetime
import time
import unittest
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
import json


  
def get_dbutils(spark):
    try:
        from pyspark.dbutils import DBUtils
        dbutils = DBUtils(spark)
    except ImportError:
        import IPython
        dbutils = IPython.get_ipython().user_ns["dbutils"]
    return dbutils
  
TESTDATA_INSERT = """[{"key":"AAAABoMIMzAwMg==","value":"{\\"before\\":null,\\"after\\":{\\"id\\":\\"3002\\",\\"username\\":\\"3002\\",\\"created_on\\":\\"2021-03-02T19:50:29.363+0000\\",\\"test\\":\\"1\\",\\"test3\\":\\"2\\",\\"t\\":\\"3\\",\\"pink\\":null,\\"pink2\\":null,\\"test10\\":null,\\"test11\\":null,\\"test12\\":null},\\"source\\":{\\"version\\":\\"0.10.0.Final\\",\\"connector\\":\\"postgresql\\",\\"name\\":\\"temporal\\",\\"ts_ms\\":1614714629363,\\"snapshot\\":\\"false\\",\\"db\\":\\"test\\",\\"schema\\":\\"public\\",\\"table\\":\\"jb\\",\\"txId\\":30408688,\\"lsn\\":484660223888,\\"xmin\\":null},\\"op\\":\\"c\\",\\"ts_ms\\":1614714629363}","topic":"temporal.public.jb","partition":"8","offset":"1","timestamp":"2021-03-02T19:50:29.745+0000","timestampType":"0"},
{"key":"AAAABoMIMzAwMQ==","value":"{\\"before\\":null,\\"after\\":{\\"id\\":\\"3001\\",\\"username\\":\\"3001\\",\\"created_on\\":\\"2021-03-02T19:49:56.888+0000\\",\\"test\\":\\"1\\",\\"test3\\":\\"2\\",\\"t\\":\\"3\\",\\"pink\\":null,\\"pink2\\":null,\\"test10\\":null,\\"test11\\":null,\\"test12\\":null},\\"source\\":{\\"version\\":\\"0.10.0.Final\\",\\"connector\\":\\"postgresql\\",\\"name\\":\\"temporal\\",\\"ts_ms\\":1614714596888,\\"snapshot\\":\\"false\\",\\"db\\":\\"test\\",\\"schema\\":\\"public\\",\\"table\\":\\"jb\\",\\"txId\\":30408687,\\"lsn\\":484660223568,\\"xmin\\":null},\\"op\\":\\"c\\",\\"ts_ms\\":1614714596888}","topic":"temporal.public.jb","partition":"9","offset":"2","timestamp":"2021-03-02T19:49:57.245+0000","timestampType":"0"}]
"""

TESTDATA_UPDATE = """[{"key":"AAAABoMIMzAwMQ==","value":"{\\"before\\":{\\"id\\":\\"3002\\",\\"username\\":\\"__debezium_unavailable_value\\",\\"created_on\\":\\"1970-01-01T00:00:00.000+0000\\",\\"test\\":\\"__debezium_unavailable_value\\",\\"test3\\":\\"__debezium_unavailable_value\\",\\"t\\":\\"__debezium_unavailable_value\\",\\"pink\\":null,\\"pink2\\":null,\\"test10\\":null,\\"test11\\":null,\\"test12\\":null},\\"after\\":null,\\"source\\":{\\"version\\":\\"0.10.0.Final\\",\\"connector\\":\\"postgresql\\",\\"name\\":\\"temporal\\",\\"ts_ms\\":1614714660395,\\"snapshot\\":\\"false\\",\\"db\\":\\"test\\",\\"schema\\":\\"public\\",\\"table\\":\\"jb\\",\\"txId\\":30408689,\\"lsn\\":484660224088,\\"xmin\\":null},\\"op\\":\\"d\\",\\"ts_ms\\":1614714660395}","topic":"temporal.public.jb","partition":"9","offset":"3","timestamp":"2021-03-02T19:51:00.745+0000","timestampType":"0"},
{"key":"AAAABoMIMzAwMQ==","value":"{\\"before\\":{\\"id\\":\\"3001\\",\\"username\\":\\"__debezium_unavailable_value\\",\\"created_on\\":\\"1970-01-01T00:00:00.000+0000\\",\\"test\\":\\"__debezium_unavailable_value\\",\\"test3\\":\\"__debezium_unavailable_value\\",\\"t\\":\\"__debezium_unavailable_value\\",\\"pink\\":null,\\"pink2\\":null,\\"test10\\":null,\\"test11\\":null,\\"test12\\":null},\\"after\\":{\\"id\\":\\"3001\\",\\"username\\":\\"9999\\",\\"created_on\\":\\"2021-03-02T19:51:16.275+0000\\",\\"test\\":\\"1\\",\\"test3\\":\\"2\\",\\"t\\":\\"3\\",\\"pink\\":null,\\"pink2\\":null,\\"test10\\":null,\\"test11\\":null,\\"test12\\":null},\\"source\\":{\\"version\\":\\"0.10.0.Final\\",\\"connector\\":\\"postgresql\\",\\"name\\":\\"temporal\\",\\"ts_ms\\":1614714723865,\\"snapshot\\":\\"false\\",\\"db\\":\\"test\\",\\"schema\\":\\"public\\",\\"table\\":\\"jb\\",\\"txId\\":30408691,\\"lsn\\":484676996640,\\"xmin\\":null},\\"op\\":\\"u\\",\\"ts_ms\\":1614714723865}","topic":"temporal.public.jb","partition":"9","offset":"6","timestamp":"2021-03-02T19:52:04.244+0000","timestampType":"0"}]
"""

JSON_SCHEMA = '{"fields":[{"metadata":{},"name":"after","nullable":true,"type":{"fields":[{"metadata":{},"name":"created_on","nullable":true,"type":"string"},{"metadata":{},"name":"id","nullable":true,"type":"string"},{"metadata":{},"name":"pink","nullable":true,"type":"string"},{"metadata":{},"name":"pink2","nullable":true,"type":"string"},{"metadata":{},"name":"t","nullable":true,"type":"string"},{"metadata":{},"name":"test","nullable":true,"type":"string"},{"metadata":{},"name":"test10","nullable":true,"type":"string"},{"metadata":{},"name":"test11","nullable":true,"type":"string"},{"metadata":{},"name":"test12","nullable":true,"type":"string"},{"metadata":{},"name":"test3","nullable":true,"type":"string"},{"metadata":{},"name":"username","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"before","nullable":true,"type":{"fields":[{"metadata":{},"name":"created_on","nullable":true,"type":"string"},{"metadata":{},"name":"id","nullable":true,"type":"string"},{"metadata":{},"name":"pink","nullable":true,"type":"string"},{"metadata":{},"name":"pink2","nullable":true,"type":"string"},{"metadata":{},"name":"t","nullable":true,"type":"string"},{"metadata":{},"name":"test","nullable":true,"type":"string"},{"metadata":{},"name":"test10","nullable":true,"type":"string"},{"metadata":{},"name":"test11","nullable":true,"type":"string"},{"metadata":{},"name":"test12","nullable":true,"type":"string"},{"metadata":{},"name":"test3","nullable":true,"type":"string"},{"metadata":{},"name":"username","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"op","nullable":true,"type":"string"},{"metadata":{},"name":"source","nullable":true,"type":{"fields":[{"metadata":{},"name":"connector","nullable":true,"type":"string"},{"metadata":{},"name":"db","nullable":true,"type":"string"},{"metadata":{},"name":"lsn","nullable":true,"type":"long"},{"metadata":{},"name":"name","nullable":true,"type":"string"},{"metadata":{},"name":"schema","nullable":true,"type":"string"},{"metadata":{},"name":"snapshot","nullable":true,"type":"string"},{"metadata":{},"name":"table","nullable":true,"type":"string"},{"metadata":{},"name":"ts_ms","nullable":true,"type":"long"},{"metadata":{},"name":"txId","nullable":true,"type":"long"},{"metadata":{},"name":"version","nullable":true,"type":"string"},{"metadata":{},"name":"xmin","nullable":true,"type":"string"}],"type":"struct"}},{"metadata":{},"name":"ts_ms","nullable":true,"type":"long"}],"type":"struct"}'


class PipelineUnitTestCases(unittest.TestCase, TestPipelineJob):

    folder = ""
    sparkCls = ""

    @staticmethod
    def toMap(path,folder):
        data = {"partition_column": None, "zorder_column": "id", "primary_key": "id",
                "partition_column_derive_from_column": None, "table_name": "jb", "reference_index": 0,
                "topic_pattern": path, "table_exists": False, "is_pattern": False, "topic_prefix": "temporal.public.",
                "database": "database_test_%s" % folder}
        record = {data["table_name"]: data}
        return record

    @staticmethod
    def current_milli_time():
        return round(time.time() * 1000)

    @classmethod
    def setUpClass(cls):
        cls.folder = str(cls.current_milli_time())
        print(cls.folder)
        cls.sparkCls = SparkSession.builder.getOrCreate()
        # cls.sparkCls = SparkSession.builder.master("local[1]").getOrCreate()

    def setUp(self):
        self.spark = self.sparkCls
        path = "dbfs:/tmp/database_test_%s" % self.folder
        global_variables = Global_variables()
        global_variables.sc = self.spark.sparkContext
        global_variables.spark = self.spark
        global_variables.schemaRegistryUrl = "schemaRegistryAddress"

        global_variables.batchOutputPath = path
        global_variables.kafka_grp = "kafkaGrp"
        global_variables.OutputPathForMaterializedView = path
        global_variables.bootstrapServers = "bootstrapServers"
        global_variables.kafkaConsumerGroupForLag = "kafkaConsumerGroupForLag"
        global_variables.startDateOfPipeline = "1-03-2021"
        global_variables.timestampToStartStreaming = None
        global_variables.test_data = None
        global_variables.test_schema = None

        if global_variables.startDateOfPipeline != "":
            timestampToStartStreaming = datetime.datetime.strptime(global_variables.startDateOfPipeline, "%d-%m-%Y")
            print("timestamp: " + str(timestampToStartStreaming))
            global_variables.timestampToStartStreaming = timestampToStartStreaming
        self.test_config = {
            "output_format": "delta",
            "global_variables": global_variables,
            "mapOfTopicAndData": self.toMap(path,self.folder),
            "output_path": path
        }
        self.job = TestPipelineJob(spark=self.spark, init_conf=self.test_config)

    def test_insert(self):
        gv = self.test_config["global_variables"]
        mapOfTopicAndData = self.test_config["mapOfTopicAndData"]["jb"]
        gv.test_data = TESTDATA_INSERT
        new_schema = StructType.fromJson(json.loads(JSON_SCHEMA))
        gv.test_schema = new_schema
        print(gv.test_data)
        self.job.launch()

        output_count = (
            self.spark
                .sql("select * from {}.jb".format(mapOfTopicAndData['database']))
                .count()
        )
        output_count2 = (
            self.spark
                .sql("select * from {}.jb where id='3001' and username='3001' ".format(mapOfTopicAndData['database']))
                .count()
        )

        self.assertEqual(output_count, 2)
        self.assertEqual(output_count2, 1)

    def test_update(self):
        gv = self.test_config["global_variables"]
        mapOfTopicAndData = self.test_config["mapOfTopicAndData"]["jb"]
        gv.test_data = TESTDATA_UPDATE
        new_schema = StructType.fromJson(json.loads(JSON_SCHEMA))
        gv.test_schema = new_schema
        print(gv.test_data)
        self.job.launch()

        output_count = (
            self.spark
                .sql("select * from {}.jb".format(mapOfTopicAndData['database']))
                .count()
        )
        output_count2 = (
            self.spark
                .sql("select * from {}.jb where id='3001' and username='9999' ".format(mapOfTopicAndData['database']))
                .count()
        )

        self.assertEqual(output_count, 1)
        self.assertEqual(output_count2, 1)

    def tearDown(self):
        pass

    @classmethod
    def tearDownClass(cls):
        dbutils = get_dbutils(cls.sparkCls)
        cls.sparkCls.sql("drop database  {} cascade".format("database_test_%s" % cls.folder))
        dbutils.fs.rm("/tmp/database_test_{0}database_test_{0}".format(cls.folder), True)
        dbutils.fs.rm("/tmp/database_test_%s" % cls.folder, True)
        print("completed")

In [ ]:
p = PipelineUnitTestCases()
p.setUpClass()
p.setUp()
p.test_insert()

In [ ]:
# %sql
# describe history dev_pipeline_test_optimize.user_summary_old

In [ ]:
# %sql
# alter table dev_pipeline_test_optimize.user_summary_old add columns (last_lat1 double)

In [ ]:
# df3.write.option("overwriteSchema", "true")

In [ ]:
# %sql
# alter table dev_pipeline_test_optimize.user_summary_old drop column last_lat

In [ ]:
# %sql
# update dev_pipeline_test_optimize.user_summary_old set last_lat1 = last_lat

In [ ]:
# %sql
# truncate table dev_pipeline_test_optimize.user_summary_old

In [ ]:
# %sql
# select last_lat from dev_pipeline_test_optimize.user_summary_old order by created_at desc limit 1000 

In [ ]:
# # Add new column with default
# from pyspark.sql.functions import lit
# from pyspark.sql.functions import col, struct, when
# from pyspark.sql.types import StructType, StructField, IntegerType, LongType, StringType, FloatType, DecimalType,DoubleType

# df = spark.sql("Select * from dev_pipeline_test_optimize.user_summary_old")

# df1 = df.withColumn("last_lat", col("last_lat").cast(DecimalType(10,0)))

# df1.write.format("delta").mode("OVERWRITE").option("overwriteSchema", "true").saveAsTable("dev_pipeline_test_optimize.user_summary_old")

In [ ]:
# # Add new column with default
# from pyspark.sql.functions import lit
# from pyspark.sql.functions import col, struct, when
# from pyspark.sql.types import StructType, StructField, IntegerType, LongType, StringType, FloatType, DecimalType,DoubleType

# df = spark.sql("Select * from dev_pipeline_test_optimize.user_summary_old")

# df1 = df.withColumn("last_lat", col("last_lat").cast(DoubleType()))

# df1.write.format("delta").mode("OVERWRITE").option("overwriteSchema", "true").saveAsTable("dev_pipeline_test_optimize.user_summary_old")

In [ ]:
# %sql
# select * 
# from dev_pipeline_test_optimize.user_summary_old 
# where CAST(last_lat AS DECIMAL(10,0)) - last_lat = 0

In [ ]:
# # Add new column with default
# from pyspark.sql.functions import lit
# from pyspark.sql.functions import col, struct, when
# from pyspark.sql.types import StructType, StructField, IntegerType, LongType, StringType, FloatType, DecimalType,DoubleType

# df = spark.sql("select * from dev_pipeline_test_optimize.user_summary_old where CAST(last_lat AS DECIMAL(10,0)) - last_lat = 0 limit 1")

# df3 = df.withColumn("last_lat", col("last_lat").cast(DoubleType()))

# #df1.write.format("delta").mode("OVERWRITE").option("overwriteSchema", "true").saveAsTable("dev_pipeline_test_optimize.user_summary_old")

# display(df3)

In [ ]:
# df3.write.format("delta").mode("overwrite").saveAsTable("dev_pipeline_test_optimize.user_summary_old1")

In [ ]:
// %scala
// def merge2(spark: SparkSession, paths: Seq[String]): DataFrame = {
//     import spark.implicits._

//     spark.sparkContext.union(paths.par.map {
//       path =>
//         spark.read.parquet(path).withColumn("myField", $"myField".cast(DoubleType)).as[Double].rdd
//     }.toList).toDF
//   }

In [ ]:
# %sql
# MERGE INTO dev_pipeline_test_optimize.user_summary_old targetTable
#               USING dev_pipeline_test_optimize.user_summary_old updatesTable
#               ON updatesTable.city_id = targetTable.last_lat
#               WHEN NOT MATCHED THEN INSERT *

In [ ]:
# %sql
# desc dev_pipeline_test_optimize.user_summary_old1

In [ ]:
%sh

curl http://172.31.32.174:8081/subjects

In [ ]:
%sh
sudo apt-get install kafkacat

In [ ]:
%sh
kafkacat -b 172.31.47.188:9092 -t test-topic

In [ ]:
# import pyspark
# from pyspark.sql import SparkSession
# from pyspark.sql.types import StructType,StructField, StringType, IntegerType,ArrayType,MapType
# from pyspark.sql.functions import col,struct,when

# spark = SparkSession.builder.master("local[1]") \
#                     .appName('SparkByExamples.com') \
#                     .getOrCreate()

# data = [("James","","Smith","36636","M",3000),
#     ("Michael","Rose","","40288","M",4000),
#     ("Robert","","Williams","42114","M",4000),
#     ("Maria","Anne","Jones","39192","F",4000),
#     ("Jen","Mary","Brown","","F",-1)
#   ]

# schema = StructType([ 
#     StructField("firstname",StringType(),True), 
#     StructField("middlename",StringType(),True), 
#     StructField("lastname",StringType(),True), 
#     StructField("id", StringType(), True), 
#     StructField("gender", StringType(), True), 
#     StructField("salary", IntegerType(), True) 
#   ])
 
# df = spark.createDataFrame(data=data,schema=schema)
# df.printSchema()
# df.show(truncate=False)

# structureData = [
#     (("James","","Smith"),"36636","M",3100),
#     (("Michael","Rose",""),"40288","M",4300),
#     (("Robert","","Williams"),"42114","M",1400),
#     (("Maria","Anne","Jones"),"39192","F",5500),
#     (("Jen","Mary","Brown"),"","F",-1)
#   ]
# structureSchema = StructType([
#         StructField('name', StructType([
#              StructField('firstname', StringType(), True),
#              StructField('middlename', StringType(), True),
#              StructField('lastname', StringType(), True)
#              ])),
#          StructField('id', StringType(), True),
#          StructField('gender', StringType(), True),
#          StructField('salary', IntegerType(), True)
#          ])

# df2 = spark.createDataFrame(data=structureData,schema=structureSchema)
# df2.printSchema()
# df2.show(truncate=False)


# updatedDF = df2.withColumn("OtherInfo", 
#     struct(col("id").alias("identifier"),
#     col("gender").alias("gender"),
#     col("salary").alias("salary"),
#     when(col("salary").cast(IntegerType()) < 2000,"Low")
#       .when(col("salary").cast(IntegerType()) < 4000,"Medium")
#       .otherwise("High").alias("Salary_Grade")
#   )).drop("id","gender","salary")

# updatedDF.printSchema()
# updatedDF.show(truncate=False)


# """ Array & Map"""


# arrayStructureSchema = StructType([
#     StructField('name', StructType([
#        StructField('firstname', StringType(), True),
#        StructField('middlename', StringType(), True),
#        StructField('lastname', StringType(), True)
#        ])),
#        StructField('hobbies', ArrayType(StringType()), True),
#        StructField('properties', MapType(StringType(),StringType()), True)
#     ])

In [ ]:
# updatedDF.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable("testStruct")

In [ ]:
# %sql
# desc table testStruct